### Scraping lyric data from metal-archives.com

In [1]:
import requests
import lxml.html as lh
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def bands_on_page(url):
    headers = { 'User-Agent': 'Mozilla/5.0'
    }
    response = requests.get(url, headers=headers)
    bs = BeautifulSoup(response.text,'lxml')
    band_links = []
    for link in bs.find_all('a'):
        band_url = str(link).split('\"')[1]
        band_id = band_url.split('/')[-1][:-1]
        band_disco = f'https://www.metal-archives.com/band/discography/id/{band_id}/tab/all'
        band_links.append(band_disco)
    return band_links

def get_albums(url):
    response = requests.get(url)
    bs = BeautifulSoup(response.text, 'lxml')
#print(bs)
    album_links = []
    for a in bs.find_all('a', {'class': ['albumn', 'single']}):
        link = str(a).split('\"')[3]
        album_links.append(link)
    return(album_links)

def get_lyrics(url):
    response = requests.get(url)
    bs = BeautifulSoup(response.text, 'lxml')
    id_strings = bs.find_all('a', class_='anchor')
    ids = []
    if(len(id_strings)) == 0:
        return ([])
    for id in id_strings:
        ids.append(str(id).split('\"')[3])
    lyrics = []
    for id in ids:
        response = requests.get(f'https://www.metal-archives.com/release/ajax-view-lyrics/id/{id}')
        bs = BeautifulSoup(response.text, 'lxml')
        lyrics.append(bs)
    return lyrics

def get_clean_lyrics(lyrics):
    test_lyric = str(lyrics[0])
    lyric = lyric.split('<br/>')[3:-2]
    lyrics_cleaned = []
    for i, line in enumerate(test_lyric):
        if  line != '\n':
            lyrics_cleaned.append(line[2:])
    return lyrics_cleaned


In [3]:
band_links = []
for i in range(0, 37800, 200):
    URL = f'https://www.metal-archives.com/search/ajax-advanced/searching/bands/?bandName=&genre=&country=US&yearCreationFrom=&yearCreationTo=&bandNotes=&status=&themes=&location=&bandLabelName=&sEcho=4&iColumns=3&sColumns=&iDisplayStart={i}&iDisplayLength=37800&mDataProp_0=0&mDataProp_1=1&mDataProp_2=2&_=1692040038976'
    band_links = np.concatenate((band_links, bands_on_page(URL)))

In [172]:


albums = []
for count, band in enumerate(band_links):
    albums.append(get_albums(band))
    if count %20 == 0:
        print(count/len(band_links))
albums = np.concatenate(([], albums))

lyrics = []
for album in albums:
    print(album)